## Using GitLoader (Langchain) to load git repos.

In [1]:
from langchain.document_loaders import GitLoader
from langchain.schema import Document
import json
from typing import Iterable

## TODO
- preprocess md file separately.
- how to preprocess code files?


## Tips from [here](https://towardsdatascience.com/10-ways-to-improve-the-performance-of-retrieval-augmented-generation-systems-5fa2cee7cd5c)
- Clean your data. Make sure it is content only and no extraneous tags.
- Explore different index types. Perhaps can use keyword based search for certain items, and llamaindex based for others.
- Experiment with chunking approach. Suggest to write up individual functions and then combine them into a main file for experiments. (for loop)
- Play around with your base prompt. Once again, can use an experiment. Refer to the prompt guide [here](https://arxiv.org/abs/2302.11382)
- Do meta-data filtering. Gitloader currently doesn't consider date. But there should be a way to obtain last modified.
-  Use query routing. It's useful to have more than one index. For e.g. one that handles the summaries, one that answers flow questions etc. OR just come up with two separate chatbots.
- Look into reranking. It is one solution to the issue of discrepancy between similarity and relevance.
- Look into transforming the user's query further. [HyDE](https://github.com/texttron/hyde) takes a query, generates a hypothetical response, and then uses both for embedding look up. This can improve performance. [*note that they use GPT3, but we can simply prompt the LLM one more time.*]
- Fine-tune your **embedding model**. The embedding model might not be well-suited to our target domain. So we can improve retrieval by fine-tuning first.
- Use LLM evaluation frameworks: Langchain-Evaluate, RAGAS

In [2]:
# The original approach loads too many files. 
# Instead, focus on Tcl, Cpp files

cpp_exts = ['.cpp', 'cc', 'c++',
            'hpp', 'hh', 'h++', 'h']
other_exts = ['.tcl', '.i', '.py', '.md']
exts = cpp_exts + other_exts

loader = GitLoader(
    clone_url="https://github.com/The-OpenROAD-Project/OpenROAD",
    repo_path="./data",
    branch="master",
    file_filter = lambda file_path: any(file_path.endswith(ext) for ext in exts)
)

data = loader.load_and_split()
data[0]

Document(page_content="# Contributor Covenant Code of Conduct\n\n## Our Pledge\n\nWe as members, contributors, and leaders pledge to make participation in our\ncommunity a harassment-free experience for everyone, regardless of age, body\nsize, visible or invisible disability, ethnicity, sex characteristics, gender\nidentity and expression, level of experience, education, socio-economic status,\nnationality, personal appearance, race, religion, or sexual identity\nand orientation.\n\nWe pledge to act and interact in ways that contribute to an open, welcoming,\ndiverse, inclusive, and healthy community.\n\n## Our Standards\n\nExamples of behavior that contributes to a positive environment for our\ncommunity include:\n\n* Demonstrating empathy and kindness toward other people\n* Being respectful of differing opinions, viewpoints, and experiences\n* Giving and gracefully accepting constructive feedback\n* Accepting responsibility and apologizing to those affected by our mistakes,\n  and le

In [19]:
len(data)

7037

In [20]:
# Code is adapted from https://github.com/langchain-ai/langchain/issues/3016
def save_docs_to_jsonl(array:Iterable[Document], file_path:str)->None:
    with open(file_path, 'w') as jsonl_file:
        for doc in array:
            jsonl_file.write(doc.json() + '\n')

def load_docs_from_jsonl(file_path)->Iterable[Document]:
    array = []
    with open(file_path, 'r') as jsonl_file:
        for line in jsonl_file:
            data = json.loads(line)
            obj = Document(**data)
            array.append(obj)
    return array
    
save_docs_to_jsonl(data,'tempdata/data.jsonl')
data2=load_docs_from_jsonl('tempdata/data.jsonl')
assert len(data) == len(data2)


## Chunking Strategies (from Pinecone) [link](https://www.pinecone.io/learn/chunking-strategies)
- Fixed-size chunking. The most straightforward is to use Langchain, CharacterTextSplitter
- Content-aware chunking. Take advantage of the nature of the content and apply more sophisticated chunking
1. Naive splitting: split by `.`, `\n`
```python
text = "..." # your text<br>
docs = text.split(".")
```

2. NLTK
```python
text = "..." # your text
from langchain.text_splitter import NLTKTextSplitter
text_splitter = NLTKTextSplitter()
docs = text_splitter.split_text(text)
```

3. spaCy
```python
text = "..." # your text
from langchain.text_splitter import SpacyTextSplitter
text_splitter = SpaCyTextSplitter()
docs = text_splitter.split_text(te)


- Recursive Chunking: divide the input text into smaller chunks hierarchically and iteratively.
```python
text = "..." # your text
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 256,
    chunk_overlap  =20
)

docs = text_splitter.create_documentstext]
- Specialized Chunking: Markdown and LateX
```python
from langchain.text_splitter import MarkdownTextSplitter
markdown_text = "..."

markdown_splitter = MarkdownTextSplitter(chunk_size=100, chunk_overlap=0)
docs = markdown_splitter.create_documents([markdown_te)
`

```


```)


```

```


## Best chunk size
- Preprocessing your data. Remove tags, noise
- Select a range of chunk sizes, setup evaluation harness
- A good size of question bank is 75. [link](https://www.mattambrogi.com/posts/chunk-size-matters/)